First load some utility modules

In [5]:
from stemutils.io import *
import json

Next we define the bulk of the script that we are going to modify for each training set

In [27]:
def get_script_text(dp):
    return f'''
from stemutils.io import *

from stemseg.processing_funcs import *

def plot_vadf(ax, r1, r2, **kwargs):
    angs = np.linspace(0, 2*np.pi, 300)

    x1, y1 = np.sin(angs)*r1, np.cos(angs)*r1
    x2, y2 = np.sin(angs)*r2, np.cos(angs)*r2

    ax.plot(x1,y1, c = 'blue', lw= 3, **kwargs)
    ax.plot(x2,y2, c = 'blue', lw= 3, **kwargs)

dp = Path('{dp}')
raw_dataset = hs.load(dp,lazy=True)

sig = pxm.signals.ElectronDiffraction2D(raw_dataset)
sig.set_diffraction_calibration(0.00981)

fsig = sig.transpose()

fsig.plot()

ffsig = sig.max()

ffsig_plot = hs.plot.plot_images(ffsig, vmax = 30, label = str(dp).split('/')[-2] + ':')

ax = ffsig_plot[0]

plot_vadf(ax, 0.11, 0.95)

vadf_path = dp.redirect('VADF', 1)

vadf_path.mk()

vadf_path.exists()

vadf_path.redirect('max_pattern_vd.jpg', 0)

ax.figure.savefig(vadf_path.redirect('max_pattern_vd.jpg', 0))

circular_roi = hs.roi.CircleROI(cx = 0.0, cy= 0.0, r =0.95, r_inner = 0.11)

roi2D = circular_roi.interactive(fsig, color="blue")

annsum = np.nan_to_num(roi2D.data, 0)

vadf_fig = plt.figure()
plt.imshow(annsum.sum(axis=(0,1)),cmap='gray')
plt.xticks([])
plt.yticks([])

vadf_fig.savefig(vadf_path.redirect('vadf.jpg', 0))
'''

def job_submission_script(fpath, lpath):
    return f'''
#!/bin/bash
#$ -l h_rt=1:00:00
#$ -cwd
#$ -P e02
#$ -l m_mem_free=64G
#$ -o {lpath}
#$ -e {lpath}


module load python/epsic3.7
module load cuda/10.1
python {fpath}

'''

Now we need to create a list of file paths for the datasets you want to train on.
You can do this however you like - I like to use this walk function within a list comprehension, using the "if" to exclude things

In [28]:
fdp = Path('/dls/science/groups/imaging/ePSIC_students/Andy_Bridger/mg28034-1/processing/Merlin/Calibrated/O3_pure')

dps = [dp for dp in fdp.walk('.hdf5','binned', max_depth =1)]

In [29]:
dps

[Path('/dls/science/groups/imaging/ePSIC_students/Andy_Bridger/mg28034-1/processing/Merlin/Calibrated/O3_pure/20210305 115625/20210305_115625.hdf5'),
 Path('/dls/science/groups/imaging/ePSIC_students/Andy_Bridger/mg28034-1/processing/Merlin/Calibrated/O3_pure/20210305 133334/20210305_133334.hdf5'),
 Path('/dls/science/groups/imaging/ePSIC_students/Andy_Bridger/mg28034-1/processing/Merlin/Calibrated/O3_pure/20210305 135737/20210305_135737.hdf5'),
 Path('/dls/science/groups/imaging/ePSIC_students/Andy_Bridger/mg28034-1/processing/Merlin/Calibrated/O3_pure/20210305 140302/20210305_140302.hdf5'),
 Path('/dls/science/groups/imaging/ePSIC_students/Andy_Bridger/mg28034-1/processing/Merlin/Calibrated/O3_pure/20210305 115108/20210305_115108.hdf5'),
 Path('/dls/science/groups/imaging/ePSIC_students/Andy_Bridger/mg28034-1/processing/Merlin/Calibrated/O3_pure/20210305 124516/20210305_124516.hdf5'),
 Path('/dls/science/groups/imaging/ePSIC_students/Andy_Bridger/mg28034-1/processing/Merlin/Calibrate

Again, define where to save these scripts and their logs

In [30]:
script_path = Path('/dls/science/groups/imaging/ePSIC_students/Andy_Bridger/ClusterJobs/LoopedVADF/mg28034/O3_pure')
log_path = script_path.redirect('logs',0)

In [31]:
log_path.mk()
log_path

Path('/dls/science/groups/imaging/ePSIC_students/Andy_Bridger/ClusterJobs/LoopedVADF/mg28034/O3_pure/logs')

In [32]:
for dp in dps:
    timestamp = dp.parts[-2].replace(' ', '_')
        
    #Get our script and where we are going to save it 

    example_script = get_script_text(dp)
    script_py_path = script_path.redirect(f'{timestamp}_vadf.py', 0)
    
    #Get our submission script and where we are going to save that 
    
    script_sub_path = script_py_path.redirect(f'sub_job_{timestamp}.sh')
    script_sub = job_submission_script(script_py_path, log_path)
    
    #Save these scripts 
    
    with open(script_py_path, 'w') as f:
        f.write(example_script)

    with open(script_sub_path, 'w') as f:
        f.write(script_sub)

This is much less computationally intensive, would probably be good manners to run it on the global cluster rather than hamilton

> module load global/cluster

> for f in ./*.sh; do qsub $f; done